# Context of the notebook
This notebook was the first one used for this project. Here, the data used was not the correct one. Indeed, this data was using rate constants instead of binding affinities.

In [2]:
# Basic python libraries
import os 
import patoolib
from glob import glob 

# Biological libraries 
from biopandas.pdb import PandasPdb
from biopandas.mol2 import PandasMol2
import mol2vec
import prot2vec

# Regular DS libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

# AI libraries 
import sklearn 

# Functions

In [2]:
"""
    These functions allow to get the bond section from mol2 files and to transform them to pandas dataframes    
                                                                                                                """

def _get_bondsection(mol2_lst):
        """Returns bond section from mol2 provided as list of strings.
        Raises ValueError if data is not provided in the mol2 format."""
        started = False
        first_idx = None
        for idx, s in enumerate(mol2_lst):
            if s.startswith("@<TRIPOS>BOND"):
                first_idx = idx + 1
                started = True
            elif started and s.startswith("@<TRIPOS>"):
                last_idx_plus1 = idx
                break
        if first_idx is None:
            # Raise error when file contains no @<TRIPOS>BOND
            # (i.e. file is no mol2 file)
            raise ValueError(
                "Structural data could not be loaded. "
                "Is the input file/text in the mol2 format?"
            )
        return mol2_lst[first_idx:last_idx_plus1]

def _bondsection_to_pandas(mol2_bond_lst, col_names, col_types): # mol2_bond_lst comes from the func _get_bondsection

        df = pd.DataFrame([lst.split() for lst in mol2_bond_lst], columns=col_names)

        for i in range(df.shape[1]):
            df[col_names[i]] = df[col_names[i]].astype(col_types[i])

        return df

In [3]:
"""
    This function allows to relocate columns to the wanted place 
                                                                """

def relocate_col(df,col_name,position):
    """
        col_name should be a string 
        position should be an integer
                                      """
    col = df.pop(col_name)
    df.insert(position,col_name,col)
    return df 

# Data preprocessing

In [4]:
path = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/"
koff_index = pd.read_excel(os.path.join(path,'koff_index.xlsx'),index_col=0)

In [21]:
koff_index.head()

,uniprot ID,ligand name,koff/s-1,method,temperature,protein type,smiles,reference,cluster
pdbcode,,,,,,,,,
3hec,q16539,3hec_ligand_native_1.mol2,0.38,Surface plasmon resonance,NaN,P38α,O=C(Nc1cc(Nc2nc(-c3cnccc3)ccn2)c(C)cc1)c1ccc(C...,"Biochemistry. 2010, 49, 3611-3618",51
3heg,q16539,3heg_ligand_native_2.mol2,0.018,Surface plasmon resonance,NaN,P38α,Clc1c(C(F)(F)F)cc(NC(=O)Nc2ccc(Oc3cc(C(=O)NC)n...,"Biochemistry. 2010, 49, 3611-3618",51
NaN,q16539,1kv2_ligand_native_3.mol2,0.000008,Surface plasmon resonance,NaN,P38α,O=C(Nc1n(-c2ccc(C)cc2)nc(C(C)(C)C)c1)Nc1c2c(c(...,"Biochemistry. 2010, 49, 3611-3618",51
NaN,q16539,1kv1_ligand_native_4.mol2,0.062,Surface plasmon resonance,NaN,P38α,Clc1ccc(NC(=O)Nc2n(C)nc(C(C)(C)C)c2)cc1,"Biochemistry. 2010, 49, 3611-3618",51
3gcq,q16539,3gcq_ligand_native_5.mol2,1.695e-3/0.046,Fluorescence change,NaN,P38α,O=C(Nc1n(-c2cc(C)ccc2)nc(C(C)(C)C)c1)Nc1ccc(Nc...,"J. AM. CHEM. SOC. 2009, 131, 13286–13296",51


In [5]:
"""
    Here, we're taking the 3D structures of proteins from pdb files
                                                                        """

path_protein = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/after_md/"
all_pdb_files = glob(os.path.join(path_protein,'**','*protein.pdb'))

liste = []
ppdb = PandasPdb()

for i in range(len(all_pdb_files)):
    #print(all_pdb_files[i])
    ppdb.read_pdb(all_pdb_files[i])
    ppdb_df = ppdb.df['ATOM']
    ppdb_df['protein_id'] = i
    liste.append(ppdb_df)
    df_proteins = pd.concat(liste,axis=0,ignore_index=True)

In [6]:
df_proteins.head()

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx,protein_id
0,ATOM,1,,N,,MET,,A,1,,...,28.176,6.955,1.0,0.0,,,N,NaN,16,0
1,ATOM,2,,CA,,MET,,A,1,,...,28.967,8.187,1.0,0.0,,,C,NaN,17,0
2,ATOM,3,,CB,,MET,,A,1,,...,30.475,7.898,1.0,0.0,,,C,NaN,18,0
3,ATOM,4,,CG,,MET,,A,1,,...,31.333,9.162,1.0,0.0,,,C,NaN,19,0
4,ATOM,5,,SD,,MET,,A,1,,...,31.137,10.097,1.0,0.0,,,S,NaN,20,0


In [7]:
"""
    Here, we're taking the 3D structures of ligands from mol2 files
                                                                        """

# Download all mol2 files 
path_ligands = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/after_md/"
all_mol2_files = glob(os.path.join(path_ligands,'**','*.mol2'))

# Here we're taking only the ligand structures files 
liste = []
pmol = PandasMol2()
for i in range(len(all_mol2_files)):
    #print(all_mol2_files[i])
    pmol.read_mol2(all_mol2_files[i])
    pmol_df = pmol._df
    pmol_df['ligand_id'] = i
    liste.append(pmol_df)
    df_lig_atoms = pd.concat(liste,axis=0,ignore_index=True)

In [8]:
"""
    Here, we're taking the structures of ligand bonds from mol2 files
                                                                        """

COLUMN_NAMES = (
    "bond_id",
    "origin_atom_id",
    "target_atom_id",
    "bond_type",
    #"status_bits"
)
COLUMN_TYPES = (int, int, int, str, str)

# Download all mol2 files 
path_ligands = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/after_md/"
all_mol2_files = glob(os.path.join(path_ligands,'**','*.mol2'))

# Here we're taking only the ligand structures files 
liste = []
pmol = PandasMol2()
for i in range(len(all_mol2_files)):
    #print(all_mol2_files[i])
    pmol.read_mol2(all_mol2_files[i])
    mol2_lst = pmol.mol2_text.split('\n') # get a list containing all information within mol2 file
    _get_bondsection(mol2_lst) # to get the bond section from this file 
    df_bonds = _bondsection_to_pandas(_get_bondsection(mol2_lst),COLUMN_NAMES,COLUMN_TYPES) # to transform the bond section into a df 
    df_bonds['ligand_id'] = i
    liste.append(df_bonds)
    df_lig_bonds = pd.concat(liste,axis=0,ignore_index=True)

In [9]:
"""
    Here, we're taking the 3D structures of complexes surround by water molecules from pdb files
                                                                                                    """

path_wat = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/after_md/"
all_watpdb_files = glob(os.path.join(path_wat,'**','*WAT.pdb'))

liste = []
ppdb = PandasPdb()

prot_residue_names_arr = df_proteins['residue_name'].unique()
prot_residue_names_list = prot_residue_names_arr.tolist()

for i in range(len(all_watpdb_files)):
    #print(all_pdb_files[i])
    ppdb.read_pdb(all_watpdb_files[i])
    ppdb_df_atom = ppdb.df['ATOM']
    ppdb_df_hetatm = ppdb.df['HETATM']
    #df_wat_concat = pd.concat(objs=[itemgetter('ATOM','HETATM')(pdb.df)[0],itemgetter('ATOM','HETATM')(pdb.df)[1]],axis=0)
    df_wat_concat = pd.concat(objs=[ppdb_df_atom,ppdb_df_hetatm],axis=0)
    df_wat_concat['complex_id'] = i
    liste.append(df_wat_concat)
    df_wat_total = pd.concat(liste,axis=0,ignore_index=True)


In [10]:
"""
    Here, we're deleting water molecules to keep the 3D structures of raw complexes
                                                                                    """

water_rows = df_wat_total[df_wat_total['residue_name']=='WAT'].index
df_wat_total.drop(labels=water_rows,inplace=True,axis=0)  

In [11]:
"""
    Here, we're creating a column which makes distinction between proteins and ligands within the complex
                                                                                                            """

prot_residue_names_arr = df_proteins['residue_name'].unique()
prot_residue_names_list = prot_residue_names_arr.tolist()

df_wat_total['protein'] = df_wat_total['residue_name'].apply(lambda x : int(x in prot_residue_names_list))


In [12]:
print(df_proteins.shape,df_lig_atoms.shape,df_lig_bonds.shape,df_wat_total.shape,koff_index.shape)

(3998102, 22) (44799, 10) (46624, 5) (4042942, 23) (680, 9)


In [13]:
relocate_col(df=df_wat_total,col_name='complex_id',position=0)
relocate_col(df=df_proteins,col_name='protein_id',position=0)
relocate_col(df=df_lig_atoms,col_name='ligand_id',position=0)
relocate_col(df=df_lig_bonds,col_name='ligand_id',position=0)

,ligand_id,bond_id,origin_atom_id,target_atom_id,bond_type
0,0,1,2,1,ar
1,0,2,2,3,ar
2,0,3,2,4,ar
3,0,4,5,2,am
4,0,5,6,5,am
...,...,...,...,...,...
46619,679,41,22,27,1
46620,679,42,23,26,1
46621,679,43,23,42,1
46622,679,44,24,25,1


# Deleting unuseful columns / Dealing with missing values 

In [14]:
df_wat_total.isnull().sum()/len(df_wat_total)*100

complex_id          0.0
record_name         0.0
atom_number         0.0
blank_1             0.0
atom_name           0.0
alt_loc             0.0
residue_name        0.0
blank_2             0.0
chain_id            0.0
residue_number      0.0
insertion           0.0
blank_3             0.0
x_coord             0.0
y_coord             0.0
z_coord             0.0
occupancy           0.0
b_factor            0.0
blank_4             0.0
segment_id          0.0
element_symbol      0.0
charge            100.0
line_idx            0.0
protein             0.0
dtype: float64

In [15]:
df_wat_total.drop(columns=['blank_1','blank_2','blank_3','blank_4'],axis=1,inplace=True)
df_wat_total.drop(columns=['alt_loc','insertion','segment_id'],axis=1,inplace=True)
df_wat_total.drop(columns=['charge'],axis=1,inplace=True) # too much missing values

In [16]:
df_wat_total.isnull().sum()/len(df_wat_total)

complex_id        0.0
record_name       0.0
atom_number       0.0
atom_name         0.0
residue_name      0.0
chain_id          0.0
residue_number    0.0
x_coord           0.0
y_coord           0.0
z_coord           0.0
occupancy         0.0
b_factor          0.0
element_symbol    0.0
line_idx          0.0
protein           0.0
dtype: float64

# Feature engineering 

## Feature engineering on df_lig_bonds

In [17]:
df_lig_bonds.bond_type.unique()

array(['ar', 'am', '1', '2', '3'], dtype=object)

In [18]:
"""
    Here, the purpose is to count the number of bonds each ligand has. 
    We want to prepare aggregated df at the complex granularity to be able to join them with the df_wat_total based on complex_id and ligand_id
                                                                                                                                                """
nb_bond_types_df = df_lig_bonds.groupby(['ligand_id','bond_type']).count().reset_index()

nb_simple_bonds_df = nb_bond_types_df[nb_bond_types_df['bond_type']=='1'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_nb_simple_bonds'})
nb_double_bonds_df = nb_bond_types_df[nb_bond_types_df['bond_type']=='2'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_nb_double_bonds'})
nb_triple_bonds_df = nb_bond_types_df[nb_bond_types_df['bond_type']=='3'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_nb_triple_bonds'})
nb_ar_bonds_df = nb_bond_types_df[nb_bond_types_df['bond_type']=='ar'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_nb_ar_bonds'})
nb_am_bonds_df = nb_bond_types_df[nb_bond_types_df['bond_type']=='am'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_nb_am_bonds'})

In [19]:
"""
    Here, the purpose is to calculate the average number of bonds each ligand has. 
    We want to prepare aggregated df at the complex granularity to be able to join them with the df_wat_total based on complex_id and ligand_id
                                                                                                                                                """
mean_bond_types_df = df_lig_bonds.groupby(['ligand_id','bond_type']).count()['bond_id'] / df_lig_bonds.groupby(['ligand_id']).count()['bond_id']
mean_bond_types_df = mean_bond_types_df.reset_index()

mean_simple_bonds_df = mean_bond_types_df[mean_bond_types_df['bond_type']=='1'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_mean_simple_bonds'})
mean_double_bonds_df = mean_bond_types_df[mean_bond_types_df['bond_type']=='2'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_mean_double_bonds'})
mean_triple_bonds_df = mean_bond_types_df[mean_bond_types_df['bond_type']=='3'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_mean_triple_bonds'})
mean_ar_bonds_df = mean_bond_types_df[mean_bond_types_df['bond_type']=='ar'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_mean_ar_bonds'})
mean_am_bonds_df = mean_bond_types_df[mean_bond_types_df['bond_type']=='am'][['ligand_id','bond_id']].rename(columns={'bond_id':'lig_mean_am_bonds'})

In [20]:
simple_bonds_df['lig_nb_simple_bonds'] - simple_bonds_df['lig_mean_simple_bonds']

NameError: name 'simple_bonds_df' is not defined

In [ ]:
"""
    Here, we're making the difference between the number of bond types and the average number of bond types for each ligand
                                                                                                                            """

simple_bonds_df = nb_simple_bonds_df.merge(mean_simple_bonds_df,how='inner',on='ligand_id') #join simple bonds df 
simple_bonds_df['lig_nb_mean_simple_diff'] = simple_bonds_df['lig_nb_simple_bonds'] - simple_bonds_df['lig_mean_simple_bonds'] # addition of a new column which substracts nb and mean 

double_bonds_df = nb_double_bonds_df.merge(mean_double_bonds_df,how='inner',on='ligand_id') #join double bonds df 
double_bonds_df['lig_nb_mean_double_diff'] = double_bonds_df['lig_nb_double_bonds'] - double_bonds_df['lig_mean_double_bonds'] # addition of a new column which substracts nb and mean 

triple_bonds_df = nb_triple_bonds_df.merge(mean_triple_bonds_df,how='inner',on='ligand_id') #join triple bonds df 
triple_bonds_df['lig_nb_mean_triple_diff'] = triple_bonds_df['lig_nb_triple_bonds'] - triple_bonds_df['lig_mean_triple_bonds'] # addition of a new column which substracts nb and mean 

ar_bonds_df = nb_ar_bonds_df.merge(mean_ar_bonds_df,how='inner',on='ligand_id') #join ar bonds df 
ar_bonds_df['lig_nb_mean_ar_diff'] = ar_bonds_df['lig_nb_ar_bonds'] - ar_bonds_df['lig_mean_ar_bonds'] # addition of a new column which substracts nb and mean 

am_bonds_df = nb_am_bonds_df.merge(mean_am_bonds_df,how='inner',on='ligand_id') #join am bonds df 
am_bonds_df['lig_nb_mean_am_diff'] = am_bonds_df['lig_nb_am_bonds'] - am_bonds_df['lig_mean_am_bonds'] # addition of a new column which substracts nb and mean 

## Feature engineering on df_lig_atoms : calculate atom distances within ligands and average charges

In [ ]:
df_wat_tot_lig_xyz = df_wat_total[df_wat_total['protein']==0][['complex_id','x_coord','y_coord','z_coord']]
df_wat_tot_prot_xyz = df_wat_total[df_wat_total['protein']==1][['complex_id','x_coord','y_coord','z_coord']]

In [ ]:
distances_lig = pd.DataFrame(ppdb.distance_df(df=df_wat_tot_lig_xyz),columns=['lig_distances'])
df_wat_tot_lig_xyz = pd.concat(objs=[df_wat_tot_lig_xyz,distances_lig],axis=1)
df_wat_tot_lig_xyz_gby = df_wat_tot_lig_xyz.groupby('complex_id').mean().reset_index()

In [ ]:
df_wat_tot_lig_xyz_gby

,complex_id,x_coord,y_coord,z_coord,lig_distances
0,0,34.229800,46.244289,36.037667,67.981402
1,1,34.584286,40.273043,43.528857,68.868637
2,2,33.726254,42.982559,37.310576,66.366201
3,3,25.015342,38.785791,17.281235,49.616270
4,4,28.827145,36.434916,39.344916,61.109553
...,...,...,...,...,...
672,675,42.101290,19.174439,42.231000,62.769590
673,676,47.579104,20.136830,43.300915,67.560466
674,677,43.914130,37.187826,40.227326,70.374847
675,678,31.568583,40.839983,45.897300,69.234375


In [ ]:
distances_prot = pd.DataFrame(ppdb.distance_df(df=df_wat_tot_prot_xyz),columns=['prot_distances'])
df_wat_tot_prot_xyz = pd.concat(objs=[df_wat_tot_prot_xyz,distances_prot],axis=1)
df_wat_tot_prot_xyz_gby = df_wat_tot_prot_xyz.groupby('complex_id').mean().reset_index()

In [ ]:
df_wat_tot_prot_xyz_gby

,complex_id,x_coord,y_coord,z_coord,prot_distances
0,0,32.423296,33.935227,31.184750,57.689213
1,1,30.292923,31.865074,33.076062,56.407700
2,2,28.955521,31.178494,33.690180,55.652443
3,3,29.544050,32.504855,30.525145,54.787314
4,4,31.566406,39.848728,37.663068,64.621779
...,...,...,...,...,...
675,675,56.783309,33.622652,41.546016,80.575337
676,676,57.760873,38.154217,40.772797,82.453975
677,677,37.677171,36.990051,36.194265,65.636057
678,678,40.130551,35.137289,50.833634,75.486996


In [ ]:
"""
    There is a diffence in legnth between the two grouped by dataframes. 
    This is due to my column "protein" which does not properly split between protein and ligands. 
    Therefore, the best to do is to use the pdb files with ligand structures itself and do the same with proteins ones to avoid any problem.
                                                                                                                                                """
df_wat_tot_prot_xyz_gby.merge(df_wat_tot_lig_xyz_gby,on="complex_id",how='outer',indicator=True).query('_merge=="left_only"')

,complex_id,x_coord_x,y_coord_x,z_coord_x,prot_distances,x_coord_y,y_coord_y,z_coord_y,lig_distances,_merge
362,362,32.078255,44.474814,54.833848,80.647638,NaN,NaN,NaN,NaN,left_only
592,592,38.734802,39.905001,30.320479,64.780555,NaN,NaN,NaN,NaN,left_only
593,593,37.804713,36.824162,31.279527,62.879572,NaN,NaN,NaN,NaN,left_only


In [ ]:
df_lig_atoms

,atom_id,atom_name,x,y,z,atom_type,subst_id,subst_name,charge,ligand_id
0,1,O1,26.174,44.116,34.504,O.co2,1,MOL,-0.6667,0
1,2,P2,27.380,44.213,35.337,P.3,1,MOL,0.0000,0
2,3,O3,27.155,43.314,36.469,O.co2,1,MOL,-0.6667,0
3,4,O4,27.232,45.594,35.803,O.co2,1,MOL,-0.6667,0
4,5,N5,28.631,43.982,34.388,N.am,1,MOL,0.0000,0
...,...,...,...,...,...,...,...,...,...,...
44794,39,H39,41.205,63.837,78.820,H,1,UNK,0.0000,679
44795,40,H40,44.520,65.444,81.157,H,1,UNK,0.0000,679
44796,41,H41,41.310,61.369,83.408,H,1,UNK,0.0000,679
44797,42,H42,39.390,59.091,80.840,H,1,UNK,0.0000,679


In [ ]:
def distance_df(df, xyz=(0.00, 0.00, 0.00)):
        """Computes Euclidean distance between atoms and a 3D point.
        Parameters
        ----------
        df : DataFrame
            DataFrame containing entries in the `PandasPdb.df['ATOM']`
            or `PandasPdb.df['HETATM']` format for the
            the distance computation to the `xyz` reference coordinates.
        xyz : tuple, default: (0.00, 0.00, 0.00)
            X, Y, and Z coordinate of the reference center for the distance
            computation.
        Returns
        ---------
        pandas.Series : Pandas Series object containing the Euclidean
            distance between the atoms in the record section and `xyz`.
        """
        return np.sqrt(
            np.sum(
                df[["x", "y", "z"]].subtract(xyz, axis=1) ** 2, axis=1
            )
        )

In [ ]:
distances_lig = pd.DataFrame(distance_df(df_lig_atoms),columns=['lig_distances'])
df_lig_atoms = pd.concat(objs=[df_lig_atoms,distances_lig],axis=1)
lig_distances_df = df_lig_atoms.groupby('ligand_id').mean().reset_index()[['ligand_id','lig_distances']]

In [ ]:
lig_distances_df.head()

,ligand_id,lig_distances
0,0,67.981402
1,1,68.868637
2,2,66.366201
3,3,49.616270
4,4,61.109553


In [ ]:
lig_total_charge = df_lig_atoms.groupby('ligand_id').sum().reset_index()[['ligand_id','charge']].rename(columns={'charge':'lig_total_charge'})
lig_mean_charge = df_lig_atoms.groupby('ligand_id').mean().reset_index()[['ligand_id','charge']].rename(columns={'charge':'lig_mean_charge'})
lig_charge_df = lig_total_charge.merge(lig_mean_charge,on='ligand_id',how='inner')
lig_charge_df['lig_tot_mean_diff'] = lig_charge_df['lig_total_charge'] - lig_charge_df['lig_mean_charge']
lig_charge_df.shape

(680, 4)

In [ ]:
lig_charge_df.head()

,ligand_id,lig_total_charge,lig_mean_charge,lig_tot_mean_diff
0,0,-4.0001,-0.088891,-3.911209
1,1,-3.0000,-0.042857,-2.957143
2,2,-3.0000,-0.050847,-2.949153
3,3,0.0000,0.000000,0.000000
4,4,0.0000,0.000000,0.000000


## Feature engineering on df_prot_atoms : calculate atom distances within proteins

In [ ]:
distances_prot = pd.DataFrame(ppdb.distance_df(df_proteins),columns=['prot_distances'])
df_proteins = pd.concat(objs=[df_proteins,distances_prot],axis=1)
prot_distances_df = df_proteins.groupby('protein_id').mean().reset_index()[['protein_id','prot_distances']]

In [ ]:
prot_distances_df

,protein_id,prot_distances,prot_distances,prot_distances
0,0,57.689213,56.350318,56.350318
1,1,56.407700,55.018816,55.018816
2,2,55.652443,54.272911,54.272911
3,3,54.787314,53.490195,53.490195
4,4,64.621779,63.268205,63.268205
...,...,...,...,...
675,675,80.575337,77.980115,77.980115
676,676,82.453975,80.339802,80.339802
677,677,65.636057,64.014513,64.014513
678,678,75.486996,73.682756,73.682756


In [ ]:
def rmsd(df1, df2, s=None, invert=False):
        """Compute the Root Mean Square Deviation between molecules.
        Parameters
        ----------
        df1 : pandas.DataFrame
            DataFrame with HETATM, ATOM, and/or ANISOU entries.
        df2 : pandas.DataFrame
            Second DataFrame for RMSD computation against df1. Must have the
            same number of entries as df1.
        s : {'main chain', 'hydrogen', 'c-alpha', 'heavy', 'carbon'} or None,
            default: None
            String to specify which entries to consider. If None, considers
            all atoms for comparison.
        invert : bool, default: False
            Inverts the string query if true. For example, the setting
            `s='hydrogen', invert=True` computes the RMSD based on all
            but hydrogen atoms.
        Returns
        ---------
        rmsd : float
            Root Mean Square Deviation between df1 and df2
        """
        get_dict = PandasPdb._init_get_dict()
        if s:
            if s not in get_dict.keys():
                raise AttributeError("s must be in " "%s or None" % get_dict.keys())
            df1 = get_dict[s](df1, invert=invert)
            df2 = get_dict[s](df2, invert=invert)

        total = (
            (df1["x_coord"].values - df2["x_coord"].values) ** 2
            + (df1["y_coord"].values - df2["y_coord"].values) ** 2
            + (df1["z_coord"].values - df2["z_coord"].values) ** 2
        )
        return round((total.sum() / df1.shape[0]) ** 0.5, 4)

In [ ]:
df_wat_total.head()

,complex_id,record_name,atom_number,atom_name,residue_name,chain_id,residue_number,x_coord,y_coord,z_coord,occupancy,b_factor,element_symbol,line_idx,protein
0,0,ATOM,1,N,MET,A,1,34.751,28.176,6.955,1.0,0.0,N,1,1
1,0,ATOM,2,H1,MET,A,1,35.465,28.408,6.276,1.0,0.0,H,2,1
2,0,ATOM,3,H2,MET,A,1,33.851,28.369,6.534,1.0,0.0,H,3,1
3,0,ATOM,4,H3,MET,A,1,34.812,27.187,7.162,1.0,0.0,H,4,1
4,0,ATOM,5,CA,MET,A,1,34.914,28.967,8.187,1.0,0.0,C,5,1


In [ ]:
koff_index

,uniprot ID,ligand name,koff/s-1,method,temperature,protein type,smiles,reference,cluster
pdbcode,,,,,,,,,
3hec,q16539,3hec_ligand_native_1.mol2,0.38,Surface plasmon resonance,NaN,P38α,O=C(Nc1cc(Nc2nc(-c3cnccc3)ccn2)c(C)cc1)c1ccc(C...,"Biochemistry. 2010, 49, 3611-3618",51
3heg,q16539,3heg_ligand_native_2.mol2,0.018,Surface plasmon resonance,NaN,P38α,Clc1c(C(F)(F)F)cc(NC(=O)Nc2ccc(Oc3cc(C(=O)NC)n...,"Biochemistry. 2010, 49, 3611-3618",51
NaN,q16539,1kv2_ligand_native_3.mol2,0.000008,Surface plasmon resonance,NaN,P38α,O=C(Nc1n(-c2ccc(C)cc2)nc(C(C)(C)C)c1)Nc1c2c(c(...,"Biochemistry. 2010, 49, 3611-3618",51
NaN,q16539,1kv1_ligand_native_4.mol2,0.062,Surface plasmon resonance,NaN,P38α,Clc1ccc(NC(=O)Nc2n(C)nc(C(C)(C)C)c2)cc1,"Biochemistry. 2010, 49, 3611-3618",51
3gcq,q16539,3gcq_ligand_native_5.mol2,1.695e-3/0.046,Fluorescence change,NaN,P38α,O=C(Nc1n(-c2cc(C)ccc2)nc(C(C)(C)C)c1)Nc1ccc(Nc...,"J. AM. CHEM. SOC. 2009, 131, 13286–13296",51
...,...,...,...,...,...,...,...,...,...
3oxc,p03366,3oxc_ligand_native_783.mol2,0.00023,surface plasmon resonance,20.0,HIV-1 Protease,O=C(NC(C(O)C[N+H]1C(C(=O)NC(C)(C)C)CC2C(C1)CCC...,"J. Med. Chem. 2002, 45, 5430-5439",155
1hxw,p03366,1hxw_ligand_native_784.mol2,0.00216,surface plasmon resonance,20.0,HIV-1 Protease,O=C(OCc1scnc1)NC(C(O)CC(NC(=O)C(NC(=O)N(Cc1nc(...,"J. Med. Chem. 2002, 45, 5430-5439",155
1ohr,p03366,1ohr_ligand_native_785.mol2,0.00067,surface plasmon resonance,20.0,HIV-1 Protease,S(CC(NC(=O)c1c(C)c(O)ccc1)C(O)C[N+H]1C(C(=O)NC...,"J. Med. Chem. 2002, 45, 5430-5439",155


# Draft

In [2]:
def _get_atomsection(mol2_lst):
        """Returns atom section from mol2 provided as list of strings.
        Raises ValueError if data is not provided in the mol2 format."""
        started = False
        first_idx = None
        for idx, s in enumerate(mol2_lst):
            if s.startswith("@<TRIPOS>ATOM"):
                first_idx = idx + 1
                started = True
            elif started and s.startswith("@<TRIPOS>"):
                last_idx_plus1 = idx
                break
        if first_idx is None:
            # Raise error when file contains no @<TRIPOS>ATOM
            # (i.e. file is no mol2 file)
            raise ValueError(
                "Structural data could not be loaded. "
                "Is the input file/text in the mol2 format?"
            )
        return mol2_lst[first_idx:last_idx_plus1]

def _get_bondsection(mol2_lst):
        """Returns bond section from mol2 provided as list of strings.
        Raises ValueError if data is not provided in the mol2 format."""
        started = False
        first_idx = None
        for idx, s in enumerate(mol2_lst):
            if s.startswith("@<TRIPOS>BOND"):
                first_idx = idx + 1
                started = True
            elif started and s.startswith("@<TRIPOS>"):
                last_idx_plus1 = idx
                break
        if first_idx is None:
            # Raise error when file contains no @<TRIPOS>BOND
            # (i.e. file is no mol2 file)
            raise ValueError(
                "Structural data could not be loaded. "
                "Is the input file/text in the mol2 format?"
            )
        return mol2_lst[first_idx:last_idx_plus1]

def _bondsection_to_pandas(mol2_bond_lst, col_names, col_types): # mol2_bond_lst comes from the func _get_bondsection

        df = pd.DataFrame([lst.split() for lst in mol2_bond_lst], columns=col_names)

        for i in range(df.shape[1]):
            df[col_names[i]] = df[col_names[i]].astype(col_types[i])

        return df

In [147]:
path_ligands = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/after_md/"
all_mol2_files = os.path.join(path_ligands,'6un1_ligand_native_731','6un1_ligand_native_731.mol2')

pmol = PandasMol2()
pmol.read_mol2(all_mol2_files)

In [161]:
coor = pmol.df[pmol.df['atom_type']=='O.co2'][['x','y','z']].iloc[0]
distance = pmol.distance(coor)

In [163]:
pmol.df['distances'] = distance
pmol.df.head()

,atom_id,atom_name,x,y,z,atom_type,subst_id,subst_name,charge,distances
0,1,S1,41.338,66.167,79.557,S.3,1,UNK,0.0,9.359014
1,2,S2,40.302,61.789,81.216,S.3,1,UNK,0.0,4.793194
2,3,O3,37.482,57.986,81.964,O.co2,1,UNK,-0.5,0.000000
3,4,O4,38.598,58.210,83.743,O.co2,1,UNK,-0.5,2.111983
4,5,O5,45.474,62.855,78.448,O.co2,1,UNK,-0.5,9.997074


In [14]:
mol2_lst = pmol.mol2_text.split('\n')
#mol2_lst.remove('')
mol2_lst

['@<TRIPOS>MOLECULE',
 '6un1_ligand_native_731.mol2',
 '43 45 1 0 0 ',
 'SMALL',
 'USER_CHARGES',
 '',
 '',
 '@<TRIPOS>ATOM',
 '  1 S1      41.3380    66.1670    79.5570 S.3   1 UNK   0.0000 ',
 '  2 S2      40.3020    61.7890    81.2160 S.3   1 UNK   0.0000 ',
 '  3 O3      37.4820    57.9860    81.9640 O.co2 1 UNK  -0.5000 ',
 '  4 O4      38.5980    58.2100    83.7430 O.co2 1 UNK  -0.5000 ',
 '  5 O5      45.4740    62.8550    78.4480 O.co2 1 UNK  -0.5000 ',
 '  6 O6      45.9730    63.4340    80.4140 O.co2 1 UNK  -0.5000 ',
 '  7 O7      43.6180    62.7680    82.2970 O.2   1 UNK   0.0000 ',
 '  8 O8      41.7860    58.0570    81.0820 O.2   1 UNK   0.0000 ',
 '  9 O9      43.6510    60.1220    83.2400 O.3   1 UNK   0.0000 ',
 ' 10 N10     43.6300    60.9000    81.0120 N.am  1 UNK   0.0000 ',
 ' 11 N11     40.7160    59.5620    82.3870 N.am  1 UNK   0.0000 ',
 ' 12 C12     44.5280    58.9750    83.2480 C.3   1 UNK   0.0000 ',
 ' 13 C13     37.6510    61.1060    80.8100 C.3   1 UNK   

In [5]:
_get_bondsection(mol2_lst)

['  1   1  15  1   ',
 '  2   1  16  1   ',
 '  3   2  21  1   ',
 '  4   2  26  1   ',
 '  5   3  18  ar  ',
 '  6   4  18  ar  ',
 '  7   5  19  ar  ',
 '  8   6  19  ar  ',
 '  9   7  20  2   ',
 ' 10   8  22  2   ',
 ' 11   9  12  1   ',
 ' 12   9  27  1   ',
 ' 13  10  20  am  ',
 ' 14  10  27  1   ',
 ' 15  10  28  1   ',
 ' 16  11  21  1   ',
 ' 17  11  22  am  ',
 ' 18  11  23  1   ',
 ' 19  12  29  1   ',
 ' 20  12  30  1   ',
 ' 21  12  31  1   ',
 ' 22  13  26  1   ',
 ' 23  13  32  1   ',
 ' 24  13  33  1   ',
 ' 25  13  34  1   ',
 ' 26  14  26  1   ',
 ' 27  14  35  1   ',
 ' 28  14  36  1   ',
 ' 29  14  37  1   ',
 ' 30  15  17  2   ',
 ' 31  15  38  1   ',
 ' 32  16  24  2   ',
 ' 33  16  39  1   ',
 ' 34  17  24  1   ',
 ' 35  17  40  1   ',
 ' 36  18  23  1   ',
 ' 37  19  25  1   ',
 ' 38  20  25  1   ',
 ' 39  21  27  1   ',
 ' 40  21  41  1   ',
 ' 41  22  27  1   ',
 ' 42  23  26  1   ',
 ' 43  23  42  1   ',
 ' 44  24  25  1   ',
 ' 45  25  43  1   ']

In [15]:
COLUMN_NAMES = (
    "bond_id",
    "origin_atom_id",
    "target_atom_id",
    "bond_type",
    #"status_bits"
)

COLUMN_TYPES = (int, int, int, str, str)

df_bonds = _bondsection_to_pandas(_get_bondsection(mol2_lst),COLUMN_NAMES,COLUMN_TYPES)

In [16]:
df_bonds.head()

,bond_id,origin_atom_id,target_atom_id,bond_type
0,1,1,15,1
1,2,1,16,1
2,3,2,21,1
3,4,2,26,1
4,5,3,18,ar


In [8]:
pmol.df

,atom_id,atom_name,x,y,z,atom_type,subst_id,subst_name,charge
0,1,S1,41.338,66.167,79.557,S.3,1,UNK,0.0
1,2,S2,40.302,61.789,81.216,S.3,1,UNK,0.0
2,3,O3,37.482,57.986,81.964,O.co2,1,UNK,-0.5
3,4,O4,38.598,58.210,83.743,O.co2,1,UNK,-0.5
4,5,O5,45.474,62.855,78.448,O.co2,1,UNK,-0.5
5,6,O6,45.973,63.434,80.414,O.co2,1,UNK,-0.5
6,7,O7,43.618,62.768,82.297,O.2,1,UNK,0.0
7,8,O8,41.786,58.057,81.082,O.2,1,UNK,0.0
8,9,O9,43.651,60.122,83.240,O.3,1,UNK,0.0
9,10,N10,43.630,60.900,81.012,N.am,1,UNK,0.0


In [190]:
l

['BOND',
 '  1   1  15  1   ',
 '  2   1  16  1   ',
 '  3   2  21  1   ',
 '  4   2  26  1   ',
 '  5   3  18  ar  ',
 '  6   4  18  ar  ',
 '  7   5  19  ar  ',
 '  8   6  19  ar  ',
 '  9   7  20  2   ',
 ' 10   8  22  2   ',
 ' 11   9  12  1   ',
 ' 12   9  27  1   ',
 ' 13  10  20  am  ',
 ' 14  10  27  1   ',
 ' 15  10  28  1   ',
 ' 16  11  21  1   ',
 ' 17  11  22  am  ',
 ' 18  11  23  1   ',
 ' 19  12  29  1   ',
 ' 20  12  30  1   ',
 ' 21  12  31  1   ',
 ' 22  13  26  1   ',
 ' 23  13  32  1   ',
 ' 24  13  33  1   ',
 ' 25  13  34  1   ',
 ' 26  14  26  1   ',
 ' 27  14  35  1   ',
 ' 28  14  36  1   ',
 ' 29  14  37  1   ',
 ' 30  15  17  2   ',
 ' 31  15  38  1   ',
 ' 32  16  24  2   ',
 ' 33  16  39  1   ',
 ' 34  17  24  1   ',
 ' 35  17  40  1   ',
 ' 36  18  23  1   ',
 ' 37  19  25  1   ',
 ' 38  20  25  1   ',
 ' 39  21  27  1   ',
 ' 40  21  41  1   ',
 ' 41  22  27  1   ',
 ' 42  23  26  1   ',
 ' 43  23  42  1   ',
 ' 44  24  25  1   ',
 ' 45  25  43  1   ']

In [162]:
l.remove('')

In [118]:
df_wat_total['complex_id'] = df_wat_total['atom_number'].apply(lambda x : i+1 if x != 1 else i)

In [120]:
df_wat_total.head()

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx,protein,complex_id
0,ATOM,1,,N,,MET,,A,1,,...,6.955,1.0,0.0,,,N,NaN,1,1,679
1,ATOM,2,,H1,,MET,,A,1,,...,6.276,1.0,0.0,,,H,NaN,2,1,680
2,ATOM,3,,H2,,MET,,A,1,,...,6.534,1.0,0.0,,,H,NaN,3,1,680
3,ATOM,4,,H3,,MET,,A,1,,...,7.162,1.0,0.0,,,H,NaN,4,1,680
4,ATOM,5,,CA,,MET,,A,1,,...,8.187,1.0,0.0,,,C,NaN,5,1,680


In [ ]:
path_protein = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/after_md/"
test = PandasPdb()
test.read_pdb(os.path.join(path_protein,"1ebw_ligand_a017_760_dock/1ebw_ligand_a017_760_dock_box.pdb"))

In [237]:
path_protein = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/after_md/"
pdb = PandasPdb()
pdb.read_pdb(os.path.join(path_protein,"1ebw_ligand_b365_766_dock/1ebw_ligand_b365_766_dock_WAT.pdb"))

In [ ]:
# Here, we're taking only the ligand structures files

# Download all mol2 files 
path_ligands = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/initial_structure/"
all_mol2_files = glob(os.path.join(path_ligands,'**','*.mol2'))

# Here we're taking only the ligand structures files 
liste = []
ligand_names = []
pmol = PandasMol2()
for i in range(len(all_mol2_files)):
    #print(all_mol2_files[i])
    pmol.read_mol2(all_mol2_files[i])
    liste.append(pmol._df)
    ligand_names.append(pmol.code) # to get ligand names which are unique for each ligand 
    #df_ligands = pd.concat(liste,axis=0,ignore_index=True)

In [321]:
path_ligands = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/initial_structure/"
pmol = PandasMol2()
pmol.read_mol2(os.path.join(path_ligands,'6oph_ligand_3_648_dock','6oph_ligand_3_648_dock.mol2'))

In [ ]:
pmol.code

'6oph_ligand_3_648_dock.mol2'

In [ ]:
complex_id = []
for i in range(len(liste)):
    complex_id.append(liste[i]['OTHERS'].iloc[0]['entry'])

In [234]:
r = pd.DataFrame(complex_id,columns=['complex_id'])
r['complex_id'].nunique()

673

In [257]:
# Here, we're taking proteins + ligands + water molecules (ATOM + HETATM atoms)
path_wat = "C:/Users/redha.cherif_artefac/GitHub_perso/Research_project/Input/after_md/"
all_watpdb_files = glob(os.path.join(path_wat,'**','*WAT.pdb'))

liste = []
ppdb = PandasPdb()

for i in range(len(all_watpdb_files)):
    #print(all_pdb_files[i])
    ppdb.read_pdb(all_watpdb_files[i])
    ppdb_df_atom = ppdb.df['ATOM']
    ppdb_df_hetatm = ppdb.df['HETATM']
    #df_wat_concat = pd.concat(objs=[itemgetter('ATOM','HETATM')(pdb.df)[0],itemgetter('ATOM','HETATM')(pdb.df)[1]],axis=0)
    df_wat_concat = pd.concat(objs=[ppdb_df_atom,ppdb_df_hetatm],axis=0)
    liste.append(df_wat_concat)
    df_wat = pd.concat(liste,axis=0,ignore_index=True)

In [314]:
# Drop water molecules from the df 
water_rows = df_wat[df_wat['residue_name']=='WAT'].index
df_wat.drop(labels=water_rows,inplace=True,axis=0)  

In [381]:
# Create a column which makes distinction between proteins and ligands based on residue_name column

prot_residue_names_arr = df_proteins['residue_name'].unique()
prot_residue_names_list = prot_residue_names_arr.tolist()

df_wat['protein'] = df_wat['residue_name'].apply(lambda x : int(x in prot_residue_names_list))


In [420]:
# Create a unique key "complex_id" which would range from 1 to 680

nb_of_complexes = 680

df_wat['complex_id'] = df_wat['atom_number'].apply(lambda x : 2 while x > 1)

SyntaxError: invalid syntax (382809384.py, line 4)